In [1]:
IN_COLAB = 'google.colab' in str(get_ipython())
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    import sys
    sys.path.append('/content/drive/My Drive/dp_tomastik/code')
    !bash "/content/drive/My Drive/dp_tomastik/code/scripts/install_libs.sh"

import matplotlib.pyplot as plt
import torch
import os
import numpy as np
import pandas as pd
import logging
import datetime
from torchio import RandomAffine, Compose, ZNormalization

import src.dataset.oars_labels_consts as OARS_LABELS
from src.consts import DATASET_MAX_BOUNDING_BOX, DESIRE_BOUNDING_BOX_SIZE
from src.helpers.threshold_calc_helpers import get_threshold_info_df
from src.helpers.show_model_dataset_pred_preview import show_model_dataset_pred_preview
from src.dataset.get_cut_lists import get_cut_lists
from src.dataset.get_full_res_cut import get_full_res_cut
from src.dataset.get_dataset import get_dataset
from src.dataset.get_dataset_info import get_dataset_info
from src.dataset.preview_dataset import preview_dataset
from src.dataset.get_dataset_transform import get_dataset_transform
from src.model_and_training.prepare_model import prepare_model
from src.model_and_training.train_loop import train_loop
from src.model_and_training.show_model_info import show_model_info
from src.model_and_training.load_checkpoint_model_info import load_checkpoint_model_info
from src.helpers.show_cuda_usage import show_cuda_usage
from src.helpers.get_rescaled_pred import get_rescaled_preds
from src.dataset.split_dataset import split_dataset, copy_split_dataset
from src.helpers.compare_prediction_with_ground_true import compare_prediction_with_ground_true, compare_one_prediction_with_ground_true
from src.helpers.get_img_outliers_pixels import get_img_outliers_pixels
from src.helpers.get_raw_with_prediction import get_raw_with_prediction
from src.model_and_training.getters.get_device import get_device


from operator import itemgetter
from IPython.display import display, Markdown
from ipywidgets import widgets

torch.manual_seed(20)
logging.basicConfig(filename='logs/pdd_data_check.log', level=logging.DEBUG)

print('Dataset biggest bounding box wihtout spinal cord', DATASET_MAX_BOUNDING_BOX)
print('Cut target size', DESIRE_BOUNDING_BOX_SIZE)
print('Done Init')

If you use TorchIO for your research, please cite the following paper:
Pérez-García et al., TorchIO: a Python library for efficient loading,
preprocessing, augmentation and patch-based sampling of medical images
in deep learning. Credits instructions: https://torchio.readthedocs.io/#credits

Dataset biggest bounding box wihtout spinal cord [56, 177, 156]
Cut target size [72, 192, 168]
Done Init


In [2]:
TRAIN_REGISTRATION=False
TRANSFORM_REGISTRATION=False
DISPLAY_REGISTRATION=False

# PDDCA

In [3]:
def preview_3d_image(img):
    if type(img) is sitk.SimpleITK.Image:
        img = sitk.GetArrayFromImage(img)

    max_slices = img.shape[0]
    def f(slice_index):    
        plt.figure(figsize=(16, 16))
        plt.imshow(img[slice_index])
        plt.show()
        print(f"debug: {img.min()}, {img.max()}")
        print(f"debug: unique {np.unique(img[slice_index])}")

    sliceSlider = widgets.IntSlider(min=0, max=max_slices - 1, step=1, value=(max_slices - 1) / 2)
    ui = widgets.VBox([widgets.HBox([sliceSlider])])
    out = widgets.interactive_output(f, {'slice_index': sliceSlider})
    # noinspection PyTypeChecker
    display(ui, out)

In [4]:
from pathlib import Path
import SimpleITK as sitk
import nrrd
import os
import cv2
import sys

LOAD_PDDCA = False
if LOAD_PDDCA:
    # PDDCA
    d ="./data/PDDCA-1.4.1"
    pddca_dir_items = sorted([o for o in os.listdir(d) if os.path.isdir(os.path.join(d,o))])

    ignore_items = ['0522c0014', '0522c0077', '0522c0079', '0522c0147', '0522c0159', '0522c0161', '0522c0190', '0522c0226', 
                    '0522c0329', '0522c0330', '0522c0427', '0522c0433', '0522c0441', '0522c0455', '0522c0457', '0522c0479']
    print(f'Loading {len(pddca_dir_items) - len(ignore_items)} items')

    pddca_items = list() 
    for item_id in pddca_dir_items:
        if item_id in ignore_items:
            # print(f"pddca {item_id}: ignoring")
            continue

        # parsing data
        data_filepath = Path.joinpath(Path(d), f'./{item_id}/img.nrrd')
        pddca_data, header = nrrd.read(data_filepath)
        pddca_data = pddca_data.astype(np.int16)
        pddca_data = np.transpose(pddca_data, axes=[2, 0, 1]).swapaxes(-2,-1)[...,::-1]

        # parsing labels
        oar_labels = ["BrainStem", "Chiasm", "Mandible", "OpticNerve_L", "OpticNerve_R", "Parotid_L", "Parotid_R", "Submandibular_L", "Submandibular_R"]
        pddca_label = np.zeros(pddca_data.shape, dtype=np.int8)

        for OAR_INDEX, OAR_KEY in enumerate(oar_labels):
            label_filepath = Path.joinpath(Path(d), f'./{item_id}/structures/{OAR_KEY}.nrrd')
            oar_pddca_label, header = nrrd.read(label_filepath)
            oar_pddca_label = oar_pddca_label.astype(np.int8)
            oar_pddca_label = np.transpose(oar_pddca_label, axes=[2, 0, 1]).swapaxes(-2,-1)[...,::-1]
            pddca_label += oar_pddca_label*(OAR_INDEX+1)

        # appending
        pddca_items.append((pddca_data, pddca_label))
        print(f"pddca {item_id}: {pddca_data.max()}, {pddca_data.min()}, {pddca_label.max()}, {pddca_label.min()}, {pddca_data.dtype}, {pddca_label.dtype}, {pddca_data.shape}, {pddca_label.shape}")

    print('Done loading')

In [5]:
LOAD_PDDCA = False
if LOAD_PDDCA:
    item_index = 1
    pddca_data, pddca_label = pddca_items[item_index]

    max_slices = pddca_data.shape[0]
    def f(slice_index):    
        plt.figure(figsize=(20, 20))
        plt.subplot(2, 2, 1)
        plt.imshow(pddca_data[slice_index], cmap="gray")
        plt.subplot(2, 2, 2)
        plt.imshow(pddca_label[slice_index])
        plt.subplot(2, 2, 3)

        tmp_combine = np.stack((pddca_data[slice_index],) * 3, axis=-1)
        tmp_combine -= tmp_combine.min()
        tmp_combine = tmp_combine / tmp_combine.max()    
        tmp = (pddca_label[slice_index] > 1) * 1
        tmp_cond = tmp > 0
        tmp_combine[tmp_cond, 0] = tmp[tmp_cond]

        plt.imshow(tmp_combine)
        plt.show()
        print(f"debug: {pddca_data.min()}, {pddca_data.max()}")
        print(f"debug: {tmp_combine.min()}, {tmp_combine.max()}")
        print(f"debug: unique {np.unique(pddca_label[slice_index])}")

    sliceSlider = widgets.IntSlider(min=0, max=max_slices - 1, step=1, value=(max_slices - 1) / 2)
    ui = widgets.VBox([widgets.HBox([sliceSlider])])
    out = widgets.interactive_output(f, {'slice_index': sliceSlider})
    # noinspection PyTypeChecker
    display(ui, out)

# STRUCT SEG 2019

In [6]:
filter_labels = OARS_LABELS.OARS_LABELS_LIST
if OARS_LABELS.SPINAL_CORD in filter_labels:
    filter_labels.remove(OARS_LABELS.SPINAL_CORD)

full_res_dataset = get_dataset(dataset_size=50, shrink_factor=1, filter_labels=filter_labels, unify_labels=False)
full_res_dataset.to_numpy()

CUDA using 1x dataset
filtering labels
filtering labels done
parsing dataset to numpy
numpy parsing done


# Registration

In [7]:
def get_registration_transform_sitk(fixed, moving, show=True):
    """https://simpleitk.readthedocs.io/en/master/link_ImageRegistrationMethod3_docs.html"""
    def command_iteration(method):
        if (method.GetOptimizerIteration() == 0):
            print("Estimated Scales: ", method.GetOptimizerScales())
        print("{0:3} = {1:7.5f} : {2}".format(method.GetOptimizerIteration(),
                                              method.GetMetricValue(),
                                              method.GetOptimizerPosition()))

    R = sitk.ImageRegistrationMethod()
    R.SetMetricAsCorrelation()
    R.SetOptimizerAsRegularStepGradientDescent(learningRate=2.0,
                                               minStep=1e-4,
                                               numberOfIterations=500,
                                               gradientMagnitudeTolerance=1e-6)
    R.SetOptimizerScalesFromIndexShift()
    tx = sitk.CenteredTransformInitializer(fixed, moving, sitk.Similarity3DTransform())
    R.SetInitialTransform(tx)
    R.SetInterpolator(sitk.sitkLinear)
    R.AddCommand(sitk.sitkIterationEvent, lambda: command_iteration(R))

    output_transform = R.Execute(fixed, moving)

    print("-------")
    print(output_transform)
    print("Optimizer stop condition: {0}".format(R.GetOptimizerStopConditionDescription()))
    print(" Iteration: {0}".format(R.GetOptimizerIteration()))
    print(" Metric value: {0}".format(R.GetMetricValue()))

    if show:
        resampler = sitk.ResampleImageFilter()
        resampler.SetReferenceImage(fixed)
        resampler.SetInterpolator(sitk.sitkLinear)
        resampler.SetDefaultPixelValue(1)
        resampler.SetTransform(output_transform)

        out = resampler.Execute(moving)

        simg1 = sitk.Cast(sitk.RescaleIntensity(fixed), sitk.sitkUInt8)
        simg2 = sitk.Cast(sitk.RescaleIntensity(out), sitk.sitkUInt8)
        cimg = sitk.Compose(simg1, simg2, simg1 // 2. + simg2 // 2.)
        preview_3d_image(cimg)

    return output_transform

In [8]:
if TRAIN_REGISTRATION:
    fixed_data, fixed_label = pddca_items[0]
    moving_data, moving_label = full_res_dataset.get_raw_item_with_label_filter(0)

    fixed_data = fixed_data.astype(np.float32)
    moving_data = moving_data.astype(np.float32)[0]
    print(fixed_data.dtype, moving_data.dtype, fixed_data.shape, moving_data.shape)

    fixed_sitk = sitk.GetImageFromArray(fixed_data)
    moving_sitk = sitk.GetImageFromArray(moving_data)

    output_transform = get_registration_transform_sitk(fixed_sitk, moving_sitk)

In [9]:
if TRANSFORM_REGISTRATION:
    def transform(image, transform):
        ref_image = image
        interpolator = sitk.sitkNearestNeighbor
        default_value = 0
        return sitk.Resample(image, ref_image, transform, interpolator, default_value)

    fixed_label_sitk = sitk.GetImageFromArray(fixed_label)
    trans_fixed_label = transform(fixed_label_sitk, output_transform.GetInverse())
    trans_fixed_label_np = sitk.GetArrayFromImage(trans_fixed_label)

In [10]:
if DISPLAY_REGISTRATION:
    max_slices = trans_fixed_label_np.shape[0]

    def f(slice_index):
        plt.figure(figsize=(20, 20))
        plt.subplot(2, 2, 1).title.set_text("Transformed label from atlas")
        plt.imshow(trans_fixed_label_np[slice_index])
        plt.subplot(2, 2, 2).title.set_text("Dataset label")
        plt.imshow(moving_label[0, slice_index])
        plt.show()

    sliceSlider = widgets.IntSlider(min=0, max=max_slices - 1, step=1, value=(max_slices - 1) / 2)
    ui = widgets.VBox([widgets.HBox([sliceSlider])])
    out = widgets.interactive_output(f, {'slice_index': sliceSlider})
    # noinspection PyTypeChecker
    display(ui, out)

# Atlas loading

In [11]:
atlas_ri = sitk.GetArrayFromImage(sitk.ReadImage('./data/PDDCA-1.2-atlas/probabilistic_atlas/RI.mhd'))
atlas_brainstem_map = sitk.GetArrayFromImage(sitk.ReadImage('./data/PDDCA-1.2-atlas/probabilistic_atlas/brain_stem_map.mhd'))
atlas_left_parotid_map = sitk.GetArrayFromImage(sitk.ReadImage('./data/PDDCA-1.2-atlas/probabilistic_atlas/left_parotid_map.mhd'))
atlas_right_parotid_map = sitk.GetArrayFromImage(sitk.ReadImage('./data/PDDCA-1.2-atlas/probabilistic_atlas/right_parotid_map.mhd'))

atlas_ri.shape, atlas_brainstem_map.shape, atlas_left_parotid_map.shape, atlas_right_parotid_map.shape

((136, 120, 219), (136, 120, 219), (136, 120, 219), (136, 120, 219))

In [12]:
max_slices = atlas_ri.shape[0]
def f(slice_index):
    plt.figure(figsize=(30, 16))
    plt.subplot(2, 2, 1)
    plt.imshow(atlas_ri[slice_index], cmap="gray")
    plt.subplot(2, 2, 2)
    plt.imshow(atlas_brainstem_map[slice_index], cmap="gray")
    plt.subplot(2, 2, 3)
    plt.imshow(atlas_left_parotid_map[slice_index], cmap="gray")
    plt.subplot(2, 2, 4)
    plt.imshow(atlas_right_parotid_map[slice_index], cmap="gray")
    plt.show()

sliceSlider = widgets.IntSlider(min=0, max=max_slices - 1, step=1, value=(max_slices - 1) / 2)
ui = widgets.VBox([widgets.HBox([sliceSlider])])
out = widgets.interactive_output(f, {'slice_index': sliceSlider})
# noinspection PyTypeChecker
display(ui, out)

Output()

In [13]:
## TODO: register atlas to NN input, think about speeding up because of data augmentation
## TODO: implement architecture CRNF

In [14]:
atlas_ri.shape, DESIRE_BOUNDING_BOX_SIZE

((136, 120, 219), [72, 192, 168])

# UNetV3

In [15]:
# filter_labels = [OARS_LABELS.EYE_L, OARS_LABELS.EYE_R, OARS_LABELS.LENS_L, OARS_LABELS.LENS_R]
filter_labels = OARS_LABELS.OARS_LABELS_LIST
if OARS_LABELS.SPINAL_CORD in filter_labels:
    filter_labels.remove(OARS_LABELS.SPINAL_CORD)
dataset = get_dataset(shrink_factor=8, filter_labels=filter_labels)

dataset.to_numpy()
split_dataset_obj = split_dataset(dataset)

get_dataset_info(dataset, split_dataset_obj)
train_dataset, valid_dataset, test_dataset = itemgetter('train_dataset', 'valid_dataset', 'test_dataset')(split_dataset_obj)

CUDA using 8x dataset
filtering labels
filtering labels done
parsing dataset to numpy
numpy parsing done
train 40, valid_size 5, test 5, full 50
train indices [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 14, 15, 17, 18, 20, 21, 22, 23, 24, 28, 30, 31, 32, 33, 34, 35, 36, 37, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
valid indices [6, 13, 19, 25, 38]
test indices [16, 26, 27, 29, 39]


In [16]:
preview_dataset(dataset, preview_index=2, show_hist=False)

data max 3071, min -1024
label max 1, min 0


Output()

In [17]:
from src.model_and_training.unet_architecture_v3 import UNetV3
from src.model_and_training.unet_architecture_v3v1 import UNetV3v1
from src.model_and_training.unet_architecture_v2 import UNetV2

log_date = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
model_name = f'{log_date}_3d_unet_model3'

print(f'Training model with dataset MODEL3')
print(f'folder \'{model_name}\'')
model_info = prepare_model(epochs=50,
                           learning_rate=3e-4,
                           in_channels=8,
                           input_data_channels=1,
                           output_label_channels=1,
                           dropout_rate=0.2,
                           train_batch_size=1,
                           model_name=model_name,
                           train_dataset=train_dataset, 
                           valid_dataset=valid_dataset, 
                           test_dataset=test_dataset,
                           model_class=UNetV3v1) # UNetV3, UNetV3v1
show_model_info(model_info)
print('\n\n')
train_loop(model_info)
print('\n\n')

# UNetV3
# Epoch [50] train done
# Batch eval [1] loss 0.37472, dsc 0.62528
# Batch eval [2] loss 0.39908, dsc 0.60092
# Batch eval [3] loss 0.39951, dsc 0.60049
# Batch eval [4] loss 0.38327, dsc 0.61673
# Batch eval [5] loss 0.36132, dsc 0.63868
# Epoch [50] valid done
# Epoch [50] T 2145.86s, deltaT 43.58s, loss: train 0.33236, valid 0.38358, dsc: train 0.66764, valid 0.61642

# UNetV2
# Epoch [50] train done
# Batch eval [1] loss 0.23973, dsc 0.76027
# Batch eval [2] loss 0.24580, dsc 0.75420
# Batch eval [3] loss 0.25798, dsc 0.74202
# Batch eval [4] loss 0.24973, dsc 0.75027
# Batch eval [5] loss 0.26769, dsc 0.73231
# Epoch [50] valid done
# Epoch [50] T 1410.82s, deltaT 28.02s, loss: train 0.20945, valid 0.25219, dsc: train 0.79055, valid 0.74781


# UNetV3 - 20210301-115221_3d_unet_model3
# Without channel expansion
# Epoch [50] train done
# Batch eval [1] loss 0.38633, dsc 0.61367
# Batch eval [2] loss 0.37051, dsc 0.62949
# Batch eval [3] loss 0.39343, dsc 0.60657
# Batch eval [4] loss 0.37833, dsc 0.62167
# Batch eval [5] loss 0.36725, dsc 0.63275
# Epoch [50] valid done
# Epoch [50] T 2363.05s, deltaT 42.47s, loss: train 0.33946, valid 0.37917, dsc: train 0.66054, valid 0.62083
# Elapsed time 0:39:23

Training model with dataset MODEL3
folder '20210301-162727_3d_unet_model3'
DEBUG: Writing to tensorboard epoch 0, step 0


/home/nikolas/fiit-dp-thesis-code/src/model_and_training/unet_architecture_v3v1.py:282: TracerWarning: Converting a tensor to a NumPy array might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  tmp = ap3.data[0].detach().cpu().numpy()
/home/nikolas/fiit-dp-thesis-code/src/model_and_training/unet_architecture_v3v1.py:288: TracerWarning: Converting a tensor to a NumPy array might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  tmp = h.data[0].detach().cpu().numpy()


DEBUG: Writing to tensorboard epoch 0, step 0
DEBUG: Writing to tensorboard epoch 0, step 0
Model number of params: 1221604, trainable 1221604



Running training loop
DEBUG: Writing to tensorboard epoch 0, step 0
Batch train [1] loss 0.99353, dsc 0.00647
Batch train [2] loss 0.98598, dsc 0.01402
Batch train [3] loss 0.97813, dsc 0.02187
Batch train [4] loss 0.97462, dsc 0.02538
Batch train [5] loss 0.97382, dsc 0.02618
Batch train [6] loss 0.95530, dsc 0.04470
Batch train [7] loss 0.95183, dsc 0.04817
Batch train [8] loss 0.94297, dsc 0.05703
Batch train [9] loss 0.92282, dsc 0.07718
Batch train [10] loss 0.93229, dsc 0.06771
Batch train [11] loss 0.89345, dsc 0.10655
Batch train [12] loss 0.84246, dsc 0.15754
Batch train [13] loss 0.85142, dsc 0.14858
Batch train [14] loss 0.81784, dsc 0.18216
Batch train [15] loss 0.80402, dsc 0.19598
Batch train [16] loss 0.79422, dsc 0.20578
Batch train [17] loss 0.81648, dsc 0.18352
Batch train [18] loss 0.73975, dsc 0.26025
Batch train [19] loss

In [18]:
model_info

{'model': UNetV3v1(
   (dconv_down1): Sequential(
     (0): Sequential(
       (0): Conv3d(1, 8, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
       (1): BatchNorm3d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (2): ReLU(inplace=True)
       (3): Conv3d(8, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
       (4): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (5): ReLU(inplace=True)
     )
     (1): Dropout(p=0.2, inplace=False)
   )
   (cse_down1): Sequential(
     (0): Linear(in_features=16, out_features=8, bias=True)
     (1): Linear(in_features=8, out_features=16, bias=True)
     (2): Sigmoid()
   )
   (dconv_atten1): Sequential(
     (0): Conv3d(16, 1, kernel_size=(1, 1, 1), stride=(1, 1, 1))
     (1): BatchNorm3d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (2): Sigmoid()
   )
   (pool1): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mo

In [19]:
display((Markdown("## Model evaluation"),))

display((Markdown("### Train Eval"),))
show_model_dataset_pred_preview(model_info, train_dataset, max_slices=MAX_PADDING_SLICES, default_slice=90)

display((Markdown("### Valid Eval"),))
show_model_dataset_pred_preview(model_info, valid_dataset, max_slices=MAX_PADDING_SLICES, default_slice=90)

# display(Markdown("### Test Eval"))
# eval_image_dataset(test_dataset, 78, 'test_plot.png')

(<IPython.core.display.Markdown object>,)

(<IPython.core.display.Markdown object>,)

NameError: name 'MAX_PADDING_SLICES' is not defined